## I DID IT MY WAY BY FRANCKY BERGER - BLUESTACKS COORDINATES CONVERTER

 ## update of event_listner.py (2025-11-4):
-this code is now able to convert global screen coordinates to Bluestacks coordinates, taking into account window borders and title bar height.

-It works only on Windows OS because it uses win32gui library to get window position and size

-It assumes Bluestacks is running at a resolution of 540x960, but this can be adjusted in the function call.

the function make_dpi_aware() is used to avoid scaling issues on high-DPI displays using ctypes to call Windows API functions.

-The on_click function captures mouse click events and prints the converted Bluestacks coordinates along with other relevant information.

-due to the complexity of window borders and title bars, the conversion may not be perfect in all scenarios, especially with custom themes or multiple monitors with different DPI settings.

-we used a variable px_brd to ignore the title bar area when calculating Bluestacks Y coordinate in the meantime we assume a fixed title bar height of 25 pixels, which may need adjustment based on actual window decorations.


In [1]:
from pynput import mouse  # Importing keyboard and mouse listeners from the pynput library
from threading import Thread  # Importing Thread for multithreading
#import StatePredictor  # Importing a custom module for state prediction
#from CardPredictor import ExtractSlots  # Importing the ExtractSlots function from the CardPredictor module
import win32gui  # Importing win32gui for interacting with Windows GUI elements
import pygetwindow as gw

In [2]:
# remove scaling issues on high-DPI displays
# transforme pixels logiques en pixels réels
# ctypes sert à faire parler Python avec le système Windows directement.in the meantime

import ctypes
def make_dpi_aware():
    try:
        # Windows 10+ recommended API
        # Charge la DLL Windows user32.dll et donne-moi accès à ses fonctions
        user32 = ctypes.windll.user32
        # Try SetProcessDpiAwarenessContext (Windows 10)
        # hasattr(objet, "nom_attribut") vérifie si un objet possède un attribut ou une fonction.
        if hasattr(user32, "SetProcessDpiAwarenessContext"):
            # DPI_AWARENESS_CONTEXT_PER_MONITOR_AWARE_V2 = -4
            user32.SetProcessDpiAwarenessContext(ctypes.c_void_p(-4))
        else:
            # Fallback to SetProcessDPIAware (older)
            user32.SetProcessDPIAware()
    except Exception:
        pass

In [3]:
def convert_to_bluestacks_coords(global_x, global_y, bluestacks_resolution=(540, 960)):


    make_dpi_aware()
    # Find BlueStacks window handle by title
    hwnd = win32gui.FindWindow(None, "BlueStacks App Player 2")
    if not hwnd:
        raise RuntimeError("BlueStacks window not found")

    # Get BlueStacks window position and size
    win_left, win_top, win_right, win_bottom = win32gui.GetWindowRect(hwnd)
    win_height = win_bottom - win_top

    left2 , top2, right2, bottom2 = win32gui.GetClientRect(hwnd)

    window_largeur = right2 - left2
    window_hauteur = bottom2 - top2

    diff = win_bottom - bottom2  # Difference between window height and client area height


    # Convertir l'origine client (0,0) en coordonnées écran
    origin_x ,  origin_y = win32gui.ClientToScreen(hwnd, (0, 0))

    # Calculate relative position inside the window
    # before we had:
    # rel_x = global_x - win_left
    # rel_y = global_y - win_top
    # but win_left and win_top where 0,0

    rel_x = global_x - origin_x
    rel_y = global_y - origin_y   # Adjust for title bar height

    # what do rel_x and rel_y represent here
    # Clamp relative position to window bounds
    # rel_x and rel_y should be between 0 and window_largeur/window_hauteur
    rel_x = max(0, min(rel_x, window_largeur))
    rel_y = max(0, min(rel_y, window_hauteur))


    virtual_w, virtual_h = bluestacks_resolution

    px_brd = 36.4 # pixels border to ignore the title bar and borders

    bs_x = round((rel_x * virtual_w / window_largeur), 2)
    # i need two integer after the floating point for more precision

    bs_y = round( (rel_y - px_brd)* virtual_h / (window_hauteur - px_brd), 2 )
    bs_y = max(0,bs_y)  # ignore title bar area

    return bs_x, bs_y,diff


def on_click(x, y, button, pressed):
        """
        Handles mouse click events and converts the coordinates to BlueStacks coordinates.

        :param x: Global X coordinate of the mouse click.
        :param y: Global Y coordinate of the mouse click.
        :param button: The mouse button that was clicked.
        :param pressed: Boolean indicating whether the button was pressed.
        """
        if pressed:
            windows = gw.getWindowsWithTitle("BlueStacks App Player 2")
            if not windows:
                raise RuntimeError("BlueStacks window not found.")
            window = windows[0]
            X, Y, dif = convert_to_bluestacks_coords(x, y,(540, 960))
            print('-' * 20)
            print(f"window resolution: {window.width}x{window.height}")
            print(f"bluestacks_x: {X}, bluestacks_y: {Y}")
            print(f"difference (window - client area) in height: {dif} pixels")
            print('-' * 20)



In [ ]:
         # Create mouse and keyboard listeners
mouse_listener = mouse.Listener(on_click=on_click)

def start_mouse():

        mouse_listener.start()
        mouse_listener.join()

    # Run listeners in separate threads
mouse_thread = Thread(target=start_mouse)

mouse_thread.start()

try:
        # Wait for both threads to complete
        mouse_thread.join()
        #keyboard_thread.join()
except KeyboardInterrupt:
        # Stop listeners on keyboard interrupt
        mouse_listener.stop()

        print("Listeners stopped.")

--------------------
window resolution: 474x816
bluestacks_x: 250.63, bluestacks_y: 960.0
difference (window - client area) in height: 0 pixels
--------------------
